In [1]:
from theano import shared
import theano.tensor as T
from theano import function

## 用共享变量创建共享函数

In [2]:
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

这段代码引入了一些新的概念。共享函数(shared function)构建了所谓的共享变量(shared variables)。这些混合的符号和非符号变量的值可以被多个函数共享。共享变量可以被用于符号表达式，就像dmatrices(...)返回的对象。同时，共享变量还拥有一个内部值，所有使用这个共享变量的函数都共享这个值。之所以称其维共享变量，是因为共享变量的值在多个函数之间共享。可以通过.get_value()方法和.set_value()方法来访问和设置共享变量的值。

这段代码引入的另一个概念是，function的updates参数。updates参数必须通过成对形式的列表（shared-variable, new expression）或字典（shared-variable作为键，new expression作为值）设置。无论以哪种方式设置，一旦执行这个function，每个共享变量shared-variable的值都将被对应的new expression的结果更新。上边的代码中，accumulator函数用state和increment amount的和取代了state的值。


## 调用函数并查看共享变量的值

In [3]:
state.get_value()

array(0)

In [4]:
# 返回旧值
accumulator(1)

array(0)

In [5]:
# 查看state的更新后的值
state.get_value()

array(1)

In [6]:
accumulator(300)

array(1)

In [7]:
state.get_value()

array(301)

## 重置state的值

In [8]:
state.set_value(-1)
accumulator(3)

array(-1)

In [9]:
state.get_value()

array(2)

## 创建被多个函数共享的变量

In [10]:
# 创建使用共享变量state的另一个函数decrementor
decrementor = function([inc], state, updates=[(state, state-inc)])
decrementor(2)

array(2)

In [11]:
state.get_value()

array(0)

为什么需要updates机制呢？你可能总是通过新表达式计算结构相似的结果，比如NumPy数组。第1个使用updates机制的原因是，它给书写代码带来便利。第2个原因（主要原因）是它所带来的执行效率。使用in-place算法可以更快地对共享变量updates。并且，theano能控制共享变量的分配（allocate），这一点是在GPU上获取高效性能的重要因素。

## 只使用共享变量而不用共享变量的值

可能存在这样的情况，你想用共享变量表达公式，但是你不想使用它的值。在这种情况下，你可以使用function的givens参数，它替换图中的特定节点以实现特定功能。

In [12]:
fn_of_state = state * 2 + inc

In [13]:
# 用来代表state值的符号变量，类型需要一致
foo = T.scalar(dtype=state.dtype)

In [14]:
# 定义function，fn_of_state中共享变量state的值用符号变量foot值替换
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])

In [15]:
skip_shared(1, 3)

array(7)

In [16]:
# 共享变量state的值并没有改变
state.get_value()

array(0)

在实践中，当你想用不同表达式替换公式中的任意部分时，可以考虑使用givens参数。